**IMPORT**

In [2]:
import pandas as pd
import json       
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas_profiling
import warnings
warnings.filterwarnings('ignore')

In [3]:
%run -i "function_m.py"

In [4]:
data = []
with open('scout_3models.json') as f:
    for line in f:
        data.append(json.loads(line))       
autos = pd.DataFrame(data)
name_columns(autos)
pd.options.display.max_columns=None

In [358]:
t_cols = ['safety_security', 'co2_emission', 'consumption', 'country_version', 
          'cylinders', 'displacement', 'make', 'model', 'model_code', 'next_inspection',
          'non_smoking_vehicle', 'body_type', 'description', "hp", 'kw', 'km', "make_model", 
          "offer_number"]

In [384]:
df = autos[t_cols]

In [360]:
df.shape

(15919, 18)

In [444]:
df.head(3)

,safety_security,co2_emission,consumption,country_version,cylinders,displacement,make,model,model_code,next_inspection,body_type,description,hp,km,offer_number
0,"[ABS, Central door lock, Daytime running light...",[\n99 g CO2/km (comb)\n],"[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",NaN,[\n3\n],"[\n1,422 cc\n]",\nAudi\n,"[\n, A1, \n]",[\n0588/BDF\n],06/2021,Sedans,"[\n, Sicherheit:, , Deaktivierung für Beifahr...",66,56013,LR-062483
1,"[ABS, Central door lock, Central door lock wit...",[\n129 g CO2/km (comb)\n],"[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",NaN,[\n4\n],"[\n1,798 cc\n]",\nAudi\n,"[\n, A1, \n]",[\n0588/BCY\n],NaN,Sedans,[\nLangstreckenfahrzeug daher die hohe Kilomet...,141,80000,NaN
2,"[ABS, Central door lock, Daytime running light...",[\n99 g CO2/km (comb)\n],"[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",NaN,NaN,"[\n1,598 cc\n]",\nAudi\n,"[\n, A1, \n]",NaN,NaN,Sedans,"[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",85,83450,AM-95365


**co2 emission**

In [385]:
df.co2_emission.isnull().sum()

1808

In [386]:
df.co2_emission.head()

0     [\n99 g CO2/km (comb)\n]
1    [\n129 g CO2/km (comb)\n]
2     [\n99 g CO2/km (comb)\n]
3     [\n99 g CO2/km (comb)\n]
4    [\n109 g CO2/km (comb)\n]
Name: co2_emission, dtype: object

In [387]:
df.co2_emission.str[0].head()

0     \n99 g CO2/km (comb)\n
1    \n129 g CO2/km (comb)\n
2     \n99 g CO2/km (comb)\n
3     \n99 g CO2/km (comb)\n
4    \n109 g CO2/km (comb)\n
Name: co2_emission, dtype: object

In [388]:
type(df.co2_emission.str[0])

pandas.core.series.Series

In [445]:
df['co2_emission'] = df.co2_emission.str[0].str.extract(r'(\d+)')

In [390]:
df.co2_emission.value_counts()[:5]

120    740
99     545
97     537
104    501
102    477
Name: co2_emission, dtype: int64

In [391]:
df.co2_emission.head()

0     99
1    129
2     99
3     99
4    109
Name: co2_emission, dtype: object

**consumption**

In [392]:
df.consumption.head()

0    [[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...
1    [[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...
2    [[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...
3    [[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...
4    [[4.1 l/100 km (comb)], [4.6 l/100 km (city)],...
Name: consumption, dtype: object

In [393]:
df.consumption[0]

[['3.8 l/100 km (comb)'], ['4.3 l/100 km (city)'], ['3.5 l/100 km (country)']]

In [394]:
df.consumption[0][0]

['3.8 l/100 km (comb)']

In [395]:
df.consumption[0][0][0]

'3.8 l/100 km (comb)'

In [396]:
df.consumption.str[0].head()

0    [3.8 l/100 km (comb)]
1    [5.6 l/100 km (comb)]
2    [3.8 l/100 km (comb)]
3    [3.8 l/100 km (comb)]
4    [4.1 l/100 km (comb)]
Name: consumption, dtype: object

In [397]:
df.consumption.str[0].str[0].head()

0    3.8 l/100 km (comb)
1    5.6 l/100 km (comb)
2    3.8 l/100 km (comb)
3    3.8 l/100 km (comb)
4    4.1 l/100 km (comb)
Name: consumption, dtype: object

In [139]:
#import re
#text='3.8 l/100 km (comb)'
#a=re.findall(r'(\d{1}\.\d{1})', text)
#print(a)

In [454]:
df['consumption_comb'] = df.consumption.str[0].str[0].str.extract(r'(\d.\d|\d)')

In [457]:
df.drop("consumption", axis=1, inplace=True)

In [399]:
df.consumption_comb.head()

0    3.8
1    5.6
2    3.8
3    3.8
4    4.1
Name: consumption_comb, dtype: object

In [ ]:
# Create a boolean for checking "combined" consumption
    #comb_bool = df.consumption.str[0].str[0].str.contains("comb", na=False)
    # Create a new column for 'consumption_comb'
    #df['consumption_comb'] = df[comb_bool].consumption.str[0].str[0].str.extract(r'(\d.\d|\d)')
    # Drop 'consumption' column
    #df.drop('consumption', axis=1, inplace=True)

**country_version**

In [400]:
df.country_version.value_counts()

[\nGermany\n]           4502
[\nItaly\n]             1038
[\nEuropean Union\n]     507
[\nNetherlands\n]        464
[\nSpain\n]              325
[\nBelgium\n]            314
[\nAustria\n]            208
[\nCzech Republic\n]      52
[\nPoland\n]              49
[\nFrance\n]              38
[\nDenmark\n]             33
[\nHungary\n]             28
[\nJapan\n]                8
[\nCroatia\n]              4
[\nSlovakia\n]             4
[\nSweden\n]               3
[\nBulgaria\n]             2
[\nRomania\n]              2
[\nSwitzerland\n]          1
[\nLuxembourg\n]           1
[\nSerbia\n]               1
[\nSlovenia\n]             1
[\nEgypt\n]                1
Name: country_version, dtype: int64

In [447]:
df['country_version'] = df.country_version.str[0].str.strip("\n")

In [402]:
df.country_version.value_counts()

Germany           4502
Italy             1038
European Union     507
Netherlands        464
Spain              325
Belgium            314
Austria            208
Czech Republic      52
Poland              49
France              38
Denmark             33
Hungary             28
Japan                8
Slovakia             4
Croatia              4
Sweden               3
Romania              2
Bulgaria             2
Luxembourg           1
Switzerland          1
Egypt                1
Slovenia             1
Serbia               1
Name: country_version, dtype: int64

**cylinders**

In [403]:
df.cylinders.value_counts()

[\n4\n]    8105
[\n3\n]    2104
[\n5\n]      22
[\n6\n]       3
[\n2\n]       2
[\n8\n]       2
[\n1\n]       1
Name: cylinders, dtype: int64

In [448]:
df['cylinders'] = df.cylinders.str[0].str.strip("\n")

In [405]:
df.cylinders.value_counts()

4    8105
3    2104
5      22
6       3
2       2
8       2
1       1
Name: cylinders, dtype: int64

**displacement**

In [406]:
df.displacement.value_counts()

[\n1,598 cc\n]     4761
[\n999 cc\n]       2438
[\n1,398 cc\n]     1314
[\n1,399 cc\n]      749
[\n1,229 cc\n]      677
[\n1,956 cc\n]      670
[\n1,461 cc\n]      595
[\n1,490 cc\n]      559
[\n1,422 cc\n]      467
[\n1,197 cc\n]      353
[\n898 cc\n]        351
[\n1,395 cc\n]      320
[\n1,968 cc\n]      301
[\n1,149 cc\n]      288
[\n1,618 cc\n]      212
[\n1,798 cc\n]      210
[\n1,498 cc\n]      196
[\n1,600 cc\n]      130
[\n1,248 cc\n]      110
[\n1,997 cc\n]      103
[\n1,364 cc\n]      102
[\n1,400 cc\n]       90
[\n998 cc\n]         72
[\n1,500 cc\n]       50
[\n2,000 cc\n]       46
[\n1,000 cc\n]       40
[\n1 cc\n]           36
[\n1,998 cc\n]       25
[\n2,480 cc\n]       20
[\n1,200 cc\n]       18
                   ... 
[\n2 cc\n]            2
[\n16,000 cc\n]       1
[\n1,856 cc\n]        1
[\n2,967 cc\n]        1
[\n1,568 cc\n]        1
[\n890 cc\n]          1
[\n1,496 cc\n]        1
[\n1,580 cc\n]        1
[\n1,195 cc\n]        1
[\n1,533 cc\n]        1
[\n1,198 cc\n]  

In [449]:
df['displacement'] = df.displacement.str[0].str.replace(",","").str.extract(r'(\d+)')

In [408]:
df.displacement.value_counts()

1598     4761
999      2438
1398     1314
1399      749
1229      677
1956      670
1461      595
1490      559
1422      467
1197      353
898       351
1395      320
1968      301
1149      288
1618      212
1798      210
1498      196
1600      130
1248      110
1997      103
1364      102
1400       90
998        72
1500       50
2000       46
1000       40
1          36
1998       25
2480       20
1200       18
         ... 
2           2
1584        1
1995        1
1696        1
1333        1
1350        1
16000       1
1100        1
1195        1
2967        1
1239        1
1686        1
1533        1
122         1
1390        1
1568        1
1800        1
1369        1
1896        1
15898       1
140         1
1580        1
890         1
1198        1
54          1
996         1
1856        1
1496        1
1368        1
973         1
Name: displacement, Length: 77, dtype: int64

**make**

In [409]:
df.make.value_counts()

\nOpel\n       7343
\nAudi\n       5712
\nRenault\n    2864
Name: make, dtype: int64

In [450]:
df['make']=df.make.str.strip('\n')

In [411]:
df.make.value_counts()

Opel       7343
Audi       5712
Renault    2864
Name: make, dtype: int64

**model**

In [412]:
df.model.value_counts()

[\n, A3, \n]          3097
[\n, A1, \n]          2614
[\n, Insignia, \n]    2598
[\n, Astra, \n]       2526
[\n, Corsa, \n]       2219
[\n, Clio, \n]        1839
[\n, Espace, \n]       991
[\n, Duster, \n]        34
[\n, A2, \n]             1
Name: model, dtype: int64

In [413]:
df.model.str[1].value_counts()

A3          3097
A1          2614
Insignia    2598
Astra       2526
Corsa       2219
Clio        1839
Espace       991
Duster        34
A2             1
Name: model, dtype: int64

In [451]:
df['model']=df.model.str[1]

In [415]:
df.model.value_counts()

A3          3097
A1          2614
Insignia    2598
Astra       2526
Corsa       2219
Clio        1839
Espace       991
Duster        34
A2             1
Name: model, dtype: int64

**model_code**

In [416]:
df.model_code.head()

0    [\n0588/BDF\n]
1    [\n0588/BCY\n]
2               NaN
3               NaN
4    [\n0588/BDF\n]
Name: model_code, dtype: object

In [452]:
df['model_code']=df.model_code.str[0].str[1:-1]

In [418]:
df.model_code.head()

0    0588/BDF
1    0588/BCY
2         NaN
3         NaN
4    0588/BDF
Name: model_code, dtype: object

**next_inspection**

In [420]:
df.next_inspection.head()

0    [\n06/2021\n, \n99 g CO2/km (comb)\n]
1                                      NaN
2                                      NaN
3                                      NaN
4                                      NaN
Name: next_inspection, dtype: object

In [421]:
df.next_inspection.sample(20)

2280                                                   NaN
12381                                                  NaN
13103                                                  NaN
4045                                                   NaN
7972                                                   NaN
10456                                                  NaN
11471    [\n08/2020\n, \n, 6.1 l/100 km (comb), \n, 7.5...
9798                                                   NaN
9456                                                   NaN
195                                                    NaN
11997                                                  NaN
6322                                                   NaN
6493                [\n07/2019\n, \n102 g CO2/km (comb)\n]
13773                                                  NaN
3563                [\n06/2021\n, \n114 g CO2/km (comb)\n]
14354                                          \n10/2022\n
9186                                                   N

In [422]:
df["next_inspection"]=df.next_inspection.str[0].str.strip("\n")

In [423]:
df.next_inspection.head()

0    06/2021
1        NaN
2        NaN
3        NaN
4        NaN
Name: next_inspection, dtype: object

**non_smoking_vehicle**

In [424]:
df.non_smoking_vehicle.sample(20)

7971                                  NaN
9465                                  NaN
6551                 [\n, \n, \nEuro 6\n]
8325                             [\n, \n]
15801                                 NaN
8862                                  NaN
13099               [\n, \n, \nEuro 6c\n]
10995                                 NaN
12697          [\n, \n, \nEuro 6d-TEMP\n]
6371                             [\n, \n]
4303                             [\n, \n]
5806                 [\n, \n, \nEuro 6\n]
6236                                  NaN
6                 [\n, \n, \n4 (Green)\n]
5996                             [\n, \n]
12375                                 NaN
13844    [\n, \n, \n92 g CO2/km (comb)\n]
2153                                  NaN
13778                                 NaN
1665                 [\n, \n, \nEuro 6\n]
Name: non_smoking_vehicle, dtype: object

In [425]:
df.drop("non_smoking_vehicle", axis=1, inplace=True)

**body_type**

In [427]:
df.body_type.sample(20)

12491    Station wagon
10401           Sedans
11416    Station wagon
13124          Compact
14385           Sedans
3196            Sedans
8510            Sedans
9017           Compact
5885     Station wagon
15646           Sedans
918            Compact
8693           Compact
4404            Sedans
15209              Van
12157    Station wagon
14680          Compact
13577          Compact
8550           Compact
7885     Station wagon
9600            Sedans
Name: body_type, dtype: object

**hp**

In [428]:
df.hp.head()

0     66 kW
1    141 kW
2     85 kW
3     66 kW
4     66 kW
Name: hp, dtype: object

In [429]:
df.hp.value_counts()

85 kW     2542
66 kW     2122
81 kW     1402
100 kW    1308
110 kW    1112
70 kW      888
125 kW     707
51 kW      695
55 kW      569
118 kW     516
92 kW      466
121 kW     392
147 kW     380
77 kW      345
56 kW      286
54 kW      276
103 kW     253
87 kW      232
165 kW     194
88 kW      177
60 kW      160
162 kW      98
- kW        88
74 kW       81
96 kW       72
71 kW       59
101 kW      47
67 kW       40
154 kW      39
122 kW      35
          ... 
80 kW        3
149 kW       3
93 kW        3
168 kW       3
98 kW        3
143 kW       2
89 kW        2
228 kW       2
63 kW        2
140 kW       2
270 kW       2
86 kW        2
150 kW       2
167 kW       2
53 kW        2
40 kW        2
127 kW       2
4 kW         1
137 kW       1
239 kW       1
195 kW       1
84 kW        1
9 kW         1
123 kW       1
44 kW        1
163 kW       1
132 kW       1
75 kW        1
133 kW       1
115 kW       1
Name: hp, Length: 81, dtype: int64

In [430]:
df["hp"]=df.hp.str.extract(r'(\d+)')

In [431]:
df.hp.value_counts()

85     2542
66     2122
81     1402
100    1308
110    1112
70      888
125     707
51      695
55      569
118     516
92      466
121     392
147     380
77      345
56      286
54      276
103     253
87      232
165     194
88      177
60      160
162      98
74       81
96       72
71       59
101      47
67       40
154      39
122      35
119      30
       ... 
168       3
80        3
149       3
93        3
98        3
228       2
53        2
143       2
63        2
40        2
270       2
86        2
167       2
140       2
150       2
127       2
89        2
133       1
115       1
75        1
44        1
195       1
163       1
4         1
239       1
132       1
84        1
9         1
123       1
137       1
Name: hp, Length: 80, dtype: int64

**km**

In [433]:
df.km.head()

0    56,013 km
1    80,000 km
2    83,450 km
3    73,000 km
4    16,200 km
Name: km, dtype: object

In [434]:
df.km.value_counts()[:20]

10 km        1045
- km         1024
1 km          367
5 km          170
50 km         148
100 km        139
15 km         109
5,000 km      104
20 km          61
3,000 km       58
12 km          56
8,000 km       49
45,000 km      48
30,000 km      47
10,000 km      47
25,000 km      47
18,000 km      46
1,000 km       46
19,000 km      44
35,000 km      44
Name: km, dtype: int64

In [435]:
df["km"]=df.km.str.replace("," , "").str.extract(r'(\d+)')

In [436]:
df.km.head()

0    56013
1    80000
2    83450
3    73000
4    16200
Name: km, dtype: object

**kw**

In [438]:
df.drop("kw", axis=1, inplace=True)

**make_model**

In [439]:
df.drop(columns = "make_model", inplace = True)

**offer_number**

In [440]:
df.offer_number.sample(20)

1115          [\n78149\n]
5124        [\nGW-V430\n]
12667    [\n8033255-Q2\n]
2707                  NaN
2091       [\n58535622\n]
13082        [\nA17001\n]
7109         [\n219307\n]
13031       [\n2252818\n]
11831      [\nM_003937\n]
8767         [\n046331\n]
6385         [\n224743\n]
13499                 NaN
3497         [\n223605\n]
13944                 NaN
3705            [\n895\n]
164          [\nV51889\n]
11347          [\n9000\n]
12754       [\n1062272\n]
2405       [\nGW-V2220\n]
3356                  NaN
Name: offer_number, dtype: object

In [441]:
df["offer_number"]=df.offer_number.str[0].str.strip("\n")

In [442]:
df.offer_number.head()

0    LR-062483
1          NaN
2     AM-95365
3          NaN
4        C1626
Name: offer_number, dtype: object

In [458]:
df.head()

,safety_security,co2_emission,country_version,cylinders,displacement,make,model,model_code,next_inspection,body_type,description,hp,km,offer_number,consumption_comb
0,"[ABS, Central door lock, Daytime running light...",99,NaN,3,1422,Audi,A1,0588/BDF,06/2021,Sedans,"[\n, Sicherheit:, , Deaktivierung für Beifahr...",66,56013,LR-062483,3.8
1,"[ABS, Central door lock, Central door lock wit...",129,NaN,4,1798,Audi,A1,0588/BCY,NaN,Sedans,[\nLangstreckenfahrzeug daher die hohe Kilomet...,141,80000,NaN,5.6
2,"[ABS, Central door lock, Daytime running light...",99,NaN,NaN,1598,Audi,A1,NaN,NaN,Sedans,"[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",85,83450,AM-95365,3.8
3,"[ABS, Alarm system, Central door lock with rem...",99,NaN,3,1422,Audi,A1,NaN,NaN,Sedans,"[\nAudi A1: , - 1e eigenaar , - Perfecte staat...",66,73000,NaN,3.8
4,"[ABS, Central door lock, Driver-side airbag, E...",109,Germany,3,1422,Audi,A1,0588/BDF,NaN,Sedans,"[\n, Technik & Sicherheit:, Xenon plus, Klimaa...",66,16200,C1626,4.1


**v_cols**

In [5]:
v_cols=['comfort_convenience', 'extras', 'drive_chain',
       'electricity_consumption', 'emission_class', 'emission_label',
       'first_registration', 'fuel', 'nr_of_doors', 'nr_of_seats',
       'other_fuel_types', 'previous_owners', 'type', 'upholstery', 'warranty',
       'weight', 'prev_owner', 'url']

In [37]:
df2=autos[v_cols]

In [38]:
df2.head(3)

,comfort_convenience,extras,drive_chain,electricity_consumption,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",[\nfront\n],NaN,[\nEuro 6\n],NaN,"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n5\n],[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",[\nfront\n],NaN,[\nEuro 6\n],[\n4 (Green)\n],"[\n, 2017, \n]","[\n, Gasoline, \n]",[\n3\n],[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",[\nfront\n],NaN,[\nEuro 6\n],[\n4 (Green)\n],"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n4\n],[\n4\n],NaN,\n1\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n99 g CO2/km (comb)\n]",NaN,1 previous owner,https://www.autoscout24.com//offers/audi-a1-sp...


**drive_chain**

In [8]:
df2.drive_chain.value_counts()

[\nfront\n]    8886
[\n4WD\n]       171
[\nrear\n]        4
Name: drive_chain, dtype: int64

In [9]:
df2.drive_chain.str[0][0]

'\nfront\n'

In [130]:
df2["drive_chain"]=df2.drive_chain.str[0].str.strip("\n")

In [11]:
df2.drive_chain.value_counts()

front    8886
4WD       171
rear        4
Name: drive_chain, dtype: int64

**electricity_consumption**

In [12]:
df2.electricity_consumption.value_counts()

[\n0 kWh/100 km (comb)\n]    137
Name: electricity_consumption, dtype: int64

In [131]:
df2.drop("electricity_consumption", axis=1, inplace=True)

**emission_class**

In [14]:
df2.emission_class.value_counts(dropna=False)

[\nEuro 6\n]          10139
NaN                    3021
[\nEuro 6d-TEMP\n]     1845
[[], [], []]            607
[\nEuro 6c\n]           127
[\nEuro 5\n]             78
[\nEuro 6d\n]            62
[\nEuro 4\n]             40
Name: emission_class, dtype: int64

In [15]:
df2.emission_class.str[0][0]

'\nEuro 6\n'

In [132]:
df2["emission_class"]=df2.emission_class.str[0].str.strip("\n")

In [17]:
df2.emission_class.value_counts(dropna=False)

Euro 6          10139
NaN              3628
Euro 6d-TEMP     1845
Euro 6c           127
Euro 5             78
Euro 6d            62
Euro 4             40
Name: emission_class, dtype: int64

In [18]:
df2.head(3)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n5\n],[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,[\n4 (Green)\n],"[\n, 2017, \n]","[\n, Gasoline, \n]",[\n3\n],[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",front,Euro 6,[\n4 (Green)\n],"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n4\n],[\n4\n],NaN,\n1\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n99 g CO2/km (comb)\n]",NaN,1 previous owner,https://www.autoscout24.com//offers/audi-a1-sp...


**emission_label**

In [19]:
df2.emission_label.value_counts()

[\n4 (Green)\n]         3553
[\n1 (No sticker)\n]     381
[[], [], []]              40
[\n5 (Blue)\n]             8
[\n3 (Yellow)\n]           2
[\n2 (Red)\n]              1
Name: emission_label, dtype: int64

In [20]:
df2.emission_label.str[0][1][4:-2]

'Green'

In [133]:
df2["emission_label"]=df2.emission_label.str[0].str[4:-2]

In [22]:
df2.emission_label.value_counts(dropna=False)

NaN           11934
Green          3553
No sticker      381
[]               40
Blue              8
Yellow            2
Red               1
Name: emission_label, dtype: int64

In [134]:
df2.loc[df2['emission_label'].notnull(), 'emission_label'] = df2.loc[df2['emission_label'].notnull(), 'emission_label'].apply(lambda y: np.nan if len(y)==0 else y)

In [24]:
df2.emission_label.value_counts(dropna=False)

NaN           11974
Green          3553
No sticker      381
Blue              8
Yellow            2
Red               1
Name: emission_label, dtype: int64

In [25]:
df2.head(3)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n5\n],[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,"[\n, 2017, \n]","[\n, Gasoline, \n]",[\n3\n],[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",front,Euro 6,Green,"[\n, 2016, \n]","[\n, Diesel (Particulate Filter), \n]",[\n4\n],[\n4\n],NaN,\n1\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n99 g CO2/km (comb)\n]",NaN,1 previous owner,https://www.autoscout24.com//offers/audi-a1-sp...


**first_registration**

In [26]:
df2.first_registration.head(3)

0    [\n, 2016, \n]
1    [\n, 2017, \n]
2    [\n, 2016, \n]
Name: first_registration, dtype: object

In [28]:
df2.first_registration.value_counts()

[\n, 2018, \n]    4522
[\n, 2016, \n]    3674
[\n, 2017, \n]    3273
[\n, 2019, \n]    2853
Name: first_registration, dtype: int64

In [34]:
df2.first_registration[0][1]

'2016'

In [44]:
df2.first_registration.str[1].value_counts()

2018    4522
2016    3674
2017    3273
2019    2853
Name: first_registration, dtype: int64

In [135]:
df2["first_registration"]=df2.first_registration.str[1]

In [46]:
df2.first_registration.value_counts()

2018    4522
2016    3674
2017    3273
2019    2853
Name: first_registration, dtype: int64

In [47]:
df2.head(3)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,"[\n, Diesel (Particulate Filter), \n]",[\n5\n],[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,"[\n, Gasoline, \n]",[\n3\n],[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",front,Euro 6,Green,2016,"[\n, Diesel (Particulate Filter), \n]",[\n4\n],[\n4\n],NaN,\n1\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n99 g CO2/km (comb)\n]",NaN,1 previous owner,https://www.autoscout24.com//offers/audi-a1-sp...


**fuel**

In [49]:
df2.fuel.value_counts()[:20]

[\n, Diesel (Particulate Filter), \n]                                                                                  4315
[\n, Super 95, \n]                                                                                                     3338
[\n, Gasoline, \n]                                                                                                     3175
[\n, Diesel, \n]                                                                                                       2984
[\n, Super 95 / Regular/Benzine 91, \n]                                                                                 424
[\n, Regular/Benzine 91, \n]                                                                                            354
[\n, Super E10 95, \n]                                                                                                  331
[\n, Super 95 (Particulate Filter), \n]                                                                                 268
[\n, Sup

In [54]:
df2.fuel.str[1][0]

'Diesel (Particulate Filter)'

In [136]:
df2["fuel"]=df2.fuel.str[1]

In [56]:
df2.fuel.head()

0    Diesel (Particulate Filter)
1                       Gasoline
2    Diesel (Particulate Filter)
3    Diesel (Particulate Filter)
4    Diesel (Particulate Filter)
Name: fuel, dtype: object

In [58]:
df2.fuel.sample(10)

11847                       Gasoline
10441       Super 95 / Super Plus 98
11213    Diesel (Particulate Filter)
504                           Diesel
10787    Diesel (Particulate Filter)
487                         Gasoline
1422                        Super 95
5576                        Super 95
9777                        Gasoline
15591                   Super E10 95
Name: fuel, dtype: object

In [59]:
df2.head(3)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),[\n5\n],[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,[\n3\n],[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",front,Euro 6,Green,2016,Diesel (Particulate Filter),[\n4\n],[\n4\n],NaN,\n1\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n99 g CO2/km (comb)\n]",NaN,1 previous owner,https://www.autoscout24.com//offers/audi-a1-sp...


**nr_of_doors**

In [60]:
df2.nr_of_doors.value_counts()

[\n5\n]    11575
[\n4\n]     3079
[\n3\n]      832
[\n2\n]      219
[\n7\n]        1
[\n1\n]        1
Name: nr_of_doors, dtype: int64

In [62]:
df2.nr_of_doors.str[0][0]

'\n5\n'

In [137]:
df2["nr_of_doors"]=df2.nr_of_doors.str[0].str.strip("\n")

In [64]:
df2.nr_of_doors.value_counts()

5    11575
4     3079
3      832
2      219
1        1
7        1
Name: nr_of_doors, dtype: int64

In [65]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,[\n5\n],NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,[\n4\n],NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...


**nr_of_seats**

In [66]:
df2.nr_of_seats.value_counts()

[\n5\n]    13336
[\n4\n]     1125
[\n7\n]      362
[\n2\n]      116
[\n6\n]        2
[\n3\n]        1
Name: nr_of_seats, dtype: int64

In [67]:
df2.nr_of_seats.str[0][0]

'\n5\n'

In [138]:
df2["nr_of_seats"]=df2.nr_of_seats.str[0].str.strip("\n")

In [70]:
df2.nr_of_seats.value_counts()

5    13336
4     1125
7      362
2      116
6        2
3        1
Name: nr_of_seats, dtype: int64

In [71]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,other_fuel_types,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,NaN,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...


**other_fuel_types**

In [72]:
df2.other_fuel_types.value_counts()

[[], [], []]    880
Name: other_fuel_types, dtype: int64

In [139]:
df2.drop("other_fuel_types", axis=1, inplace=True)

In [74]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,\n2\n,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...


**previous_owners**

In [76]:
df2.previous_owners.value_counts()[:10]

\n1\n                                                                                        8101
\n2\n                                                                                         766
\n0\n                                                                                         163
\n3\n                                                                                          17
[\n1\n, \n168 g CO2/km (comb)\n]                                                               13
[\n1\n, \n110 g CO2/km (comb)\n]                                                               12
[\n1\n, \n139 g CO2/km (comb)\n]                                                               12
[\n1\n, \n135 g CO2/km (comb)\n]                                                                9
[\n1\n, \n120 g CO2/km (comb)\n]                                                                8
[\n1\n, \n, 6.3 l/100 km (comb), \n, 7.8 l/100 km (city), \n, 5.4 l/100 km (country), \n]       7
Name: previous_owner

In [81]:
df2.previous_owners.str[1][0]

'2'

In [140]:
df2["previous_owners"]=df2.previous_owners.str[1]

In [85]:
df2.previous_owners.value_counts()[:10]

1                          8101
2                           766
0                           163
\n                           55
3                            17
\n139 g CO2/km (comb)\n      16
\n168 g CO2/km (comb)\n      13
\n110 g CO2/km (comb)\n      12
\n120 g CO2/km (comb)\n      10
\n135 g CO2/km (comb)\n       9
Name: previous_owners, dtype: int64

In [141]:
df2.loc[df2['previous_owners'].notnull(), 'previous_owners'] = df2.loc[df2['previous_owners'].notnull(), 'previous_owners'].apply(lambda y: np.nan if len(y)>1 else y)

In [87]:
df2.previous_owners.value_counts()[:10]

1     8101
2      766
0      163
\n      55
3       17
4        2
Name: previous_owners, dtype: int64

In [142]:
df2["previous_owners"]=df2.previous_owners.replace("\n", np.nan)

In [89]:
df2.previous_owners.value_counts()[:10]

1    8101
2     766
0     163
3      17
4       2
Name: previous_owners, dtype: int64

In [90]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,warranty,weight,prev_owner,url
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,2,"[, Used, , Diesel (Particulate Filter)]","[\nCloth, Black\n]","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,"[, Used, , Gasoline]","[\nCloth, Grey\n]",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...


**type**

In [93]:
df2.type.value_counts()[:10]

[, Used, , Diesel (Particulate Filter)]              3475
[, Used, , Diesel]                                   2516
[, Used, , Gasoline]                                 2367
[, Used, , Super 95]                                 1818
[, Pre-registered, , Super 95]                        500
[, New, , Super 95]                                   497
[, Employee's car, , Diesel (Particulate Filter)]     343
[, New, , Gasoline]                                   340
[, Used, , Super 95 / Regular/Benzine 91]             324
[, Employee's car, , Super 95]                        264
Name: type, dtype: int64

In [96]:
df2.type.str[1].sample(10)

7914                New
14157    Pre-registered
15457              Used
14020              Used
3990               Used
15583               New
10495              Used
13907              Used
5800               Used
10726              Used
Name: type, dtype: object

In [143]:
df2["type"]=df2.type.str[1]

In [99]:
df2.type.value_counts()[:10]

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
Name: type, dtype: int64

**upholstery**

In [106]:
df2.upholstery.value_counts()[:10]

[\nCloth, Black\n]           5821
[\nPart leather, Black\n]    1121
[\nCloth\n]                  1005
[\nCloth, Grey\n]             891
[\nCloth, Other\n]            639
[\nFull leather, Black\n]     575
[\nBlack\n]                   491
[\nGrey\n]                    273
[\nOther, Other\n]            182
[\nPart leather\n]            140
Name: upholstery, dtype: int64

In [107]:
df2.upholstery.str[0][0]

'\nCloth, Black\n'

In [144]:
df2["upholstery"]=df2.upholstery.str[0].str.strip("\n")

In [109]:
df2.upholstery.value_counts()[:10]

Cloth, Black           5821
Part leather, Black    1121
Cloth                  1005
Cloth, Grey             891
Cloth, Other            639
Full leather, Black     575
Black                   491
Grey                    273
Other, Other            182
Part leather            140
Name: upholstery, dtype: int64

In [110]:
df2.upholstery.sample(10)

135         alcantara, Black
4475                     NaN
9566            Cloth, Black
7420            Cloth, Black
7145            Cloth, Black
12595    Part leather, Black
1986                     NaN
11939           Cloth, Black
4003            Cloth, Black
3806                     NaN
Name: upholstery, dtype: object

In [111]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,warranty,weight,prev_owner,url,df2.upholstery
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,2,Used,"Cloth, Black","[\n, \n, \n4 (Green)\n]","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...,"Cloth, Black"
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,Used,"Cloth, Grey",NaN,"[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...,"Cloth, Grey"


**warranty**

In [149]:
df2.warranty.value_counts()[:10]

[\n, \n, \nEuro 6\n]           1868
\n12 months\n                  1177
\n                              979
\n24 months\n                   566
[\n, \n, \n4 (Green)\n]         517
[\n12 months\n, \nEuro 6\n]     425
[\n, \n]                        364
[\n, \n, \nEuro 6d-TEMP\n]      362
[\n12 months\n]                 128
\n48 months\n                   109
Name: warranty, dtype: int64

In [150]:
df2.warranty.str[0].sample(20)

5296                \n
3629               NaN
227                 \n
3550               NaN
5357               NaN
11768    \n12 months\n
9141                \n
9777               NaN
6420                \n
10842              NaN
9244               NaN
10050              NaN
6832                \n
12864              NaN
8734               NaN
8727               NaN
4429                \n
101                 \n
11951    \n60 months\n
5752                \n
Name: warranty, dtype: object

In [151]:
df2["warranty"]=df2.warranty.str[0].str.extract(r'(\d+\ \w{6})')

In [153]:
df2.warranty.value_counts()[:10]

12 months    1417
24 months     552
60 months     382
36 months     223
6 months       49
72 months      44
48 months      40
3 months       14
23 months      10
20 months       7
Name: warranty, dtype: int64

In [154]:
df2["warranty_months"]=df2.warranty.str.extract(r'(\d+)')
df2.drop("warranty", axis=1, inplace=True)

In [157]:
df2.warranty_months.value_counts()[:10]

12    1417
24     552
60     382
36     223
6       49
72      44
48      40
3       14
23      10
20       7
Name: warranty_months, dtype: int64

In [156]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,weight,prev_owner,url,warranty_months
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,2,Used,"Cloth, Black","[\n1,220 kg\n]",2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...,NaN
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,Used,"Cloth, Grey","[\n1,255 kg\n]",None,https://www.autoscout24.com//offers/audi-a1-1-...,NaN


**weight**

In [162]:
df2.weight.value_counts()[:50]

[\n1,163 kg\n]    574
[\n1,360 kg\n]    356
[\n1,165 kg\n]    301
[\n1,335 kg\n]    242
[\n1,135 kg\n]    213
[\n1,199 kg\n]    205
[\n1,734 kg\n]    170
[\n1,180 kg\n]    168
[\n1,503 kg\n]    165
[\n1,350 kg\n]    155
[\n1,355 kg\n]    135
[\n1,260 kg\n]    127
[\n1,275 kg\n]    112
[\n1,278 kg\n]    110
[\n1,487 kg\n]    109
[\n1,425 kg\n]    109
[\n1,255 kg\n]    108
[\n1,200 kg\n]    107
[\n1,522 kg\n]    103
[\n1,273 kg\n]    103
[\n1,280 kg\n]    102
[\n1,403 kg\n]     91
[\n1,120 kg\n]     90
[\n1,659 kg\n]     89
[\n1,195 kg\n]     89
[\n1,701 kg\n]     87
[\n1,250 kg\n]     84
[\n1,441 kg\n]     82
[\n1,285 kg\n]     80
[\n1,308 kg\n]     80
[\n1,613 kg\n]     75
[\n1,110 kg\n]     75
[\n1,279 kg\n]     72
[\n1,364 kg\n]     70
[\n1,345 kg\n]     67
[\n1,733 kg\n]     65
[\n1,071 kg\n]     64
[\n1,325 kg\n]     64
[\n1,141 kg\n]     64
[\n1,685 kg\n]     64
[\n1,230 kg\n]     63
[\n1,845 kg\n]     56
[\n1,090 kg\n]     54
[\n1,052 kg\n]     53
[\n1,664 kg\n]     52
[\n1,154 k

In [177]:
df2.weight.str[0][0]

'\n1,220 kg\n'

In [179]:
df2["weight"]=df2.weight.str[0].str.extract(r'(\d{1}\,\d{3})')

In [180]:
df2.weight.sample(5)

15018      NaN
11350    1,585
1736       NaN
4015     1,880
14625    1,119
Name: weight, dtype: object

In [181]:
df2["weight"]=df2.weight.str.replace(",","")

In [182]:
df2.weight.sample(5)

2058     1180
12547     NaN
15510     NaN
678      1640
8414      NaN
Name: weight, dtype: object

In [183]:
df2.head(2)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,weight,prev_owner,url,warranty_months
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,2,Used,"Cloth, Black",1220,2 previous owners,https://www.autoscout24.com//offers/audi-a1-sp...,NaN
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,Used,"Cloth, Grey",1255,None,https://www.autoscout24.com//offers/audi-a1-1-...,NaN


In [184]:
df2.drop("prev_owner", axis=1, inplace=True)

In [186]:
df2.head(3)

,comfort_convenience,extras,drive_chain,emission_class,emission_label,first_registration,fuel,nr_of_doors,nr_of_seats,previous_owners,type,upholstery,weight,url,warranty_months
0,"[Air conditioning, Armrest, Automatic climate ...","[Alloy wheels, Catalytic Converter, Voice Cont...",front,Euro 6,NaN,2016,Diesel (Particulate Filter),5,5,2,Used,"Cloth, Black",1220,https://www.autoscout24.com//offers/audi-a1-sp...,NaN
1,"[Air conditioning, Automatic climate control, ...","[Alloy wheels, Sport seats, Sport suspension, ...",front,Euro 6,Green,2017,Gasoline,3,4,NaN,Used,"Cloth, Grey",1255,https://www.autoscout24.com//offers/audi-a1-1-...,NaN
2,"[Air conditioning, Cruise control, Electrical ...","[Alloy wheels, Voice Control]",front,Euro 6,Green,2016,Diesel (Particulate Filter),4,4,1,Used,"Cloth, Black",NaN,https://www.autoscout24.com//offers/audi-a1-sp...,NaN
